In [1]:
import os.path
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns

from glob import glob
import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from sklearn.preprocessing import LabelEncoder

In [2]:
print(os.getcwd()) 

C:\Users\hi\Desktop\DeepLearning


In [3]:
base_dir = os.chdir(os.getcwd() + "/img-fashion")
print(os.getcwd())

C:\Users\hi\Desktop\DeepLearning\img-fashion


In [4]:
data_list = glob('*.jpg')
print('파일 개수 >> ', len(data_list))
print('파일 목록 타입 >> ', type(data_list))

파일 개수 >>  7652
파일 목록 타입 >>  <class 'list'>


In [5]:
token = text_to_word_sequence(data_list[0])
print(token)

['blouse', '1', 'jpg']


In [6]:
label = []
for x in data_list:
    token = text_to_word_sequence(x)
    label.append(token[0])
print(len(label))

7652


In [7]:
items = label
encoder = LabelEncoder()
label = encoder.fit_transform(items)
label

array([0, 0, 0, ..., 7, 7, 7], dtype=int64)

In [8]:
data_height = 150
data_width = 150
channel_n = 3

In [9]:
def make_file(dat_height, data_width, channel_n, data_list_len):
    label = []
    images = np.zeros((data_list_len, dat_height, data_width, channel_n))
    
    for i, image in enumerate(data_list):
        
        token = text_to_word_sequence(image)
        label.append(token[0])
        
        image = cv2.imread(image)
        image = cv2.resize(image, (dat_height, data_width)) / 255
        
        images[i, :, :, :] = image
    label = np.array(label)
    return (label, images)    

In [10]:
label, images = make_file(data_height, data_width, channel_n, len(data_list))

In [11]:
items = label
encoder = LabelEncoder()
label = encoder.fit_transform(items)
label, len(label) 

(array([0, 0, 0, ..., 7, 7, 7], dtype=int64), 7652)

In [12]:
x = images
y = label

X_train, X_test, y_train, y_test = train_test_split(x, 
                                                    y, 
                                                    test_size= 0.2,
                                                    random_state=30
                                                   )
print(X_train.shape, X_test.shape)

(6121, 150, 150, 3) (1531, 150, 150, 3)


In [13]:
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.optimizers import SGD

In [14]:
cnn_base = ResNet50(include_top=False, weights='imagenet',
                   input_shape=[data_height, data_width, channel_n], classes=8)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\hi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2961, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-aa21a617ab3d>", line 2, in <module>
    input_shape=[data_height, data_width, channel_n], classes=8)
  File "C:\Users\hi\Anaconda3\lib\site-packages\tensorflow_core\python\keras\applications\__init__.py", line 46, in wrapper
    return base_fun(*args, **kwargs)
  File "C:\Users\hi\Anaconda3\lib\site-packages\tensorflow_core\python\keras\applications\resnet.py", line 33, in ResNet50
    return resnet.ResNet50(*args, **kwargs)
  File "C:\Users\hi\Anaconda3\lib\site-packages\keras_applications\resnet_common.py", line 435, in ResNet50
    **kwargs)
  File "C:\Users\hi\Anaconda3\lib\site-packages\keras_applications\resnet_common.py", line 411, in ResNet
    model.load_weights(weights_path)
  File "C:\Users\hi\Anaconda3\lib\site-packages\tensorflow_core\python\keras\engine\t

AttributeError: 'str' object has no attribute 'decode'

In [ ]:
model = Sequential()

model.add(cnn_base) 
model.add(Flatten()) ## 이미지 데이터를 1차원으로 만들어주는 객체 

model.add(Dense(128, activation='relu'))
# model.add(Dropout(rate=0.3))
model.add(Dense(64, activation='relu'))
# model.add(Dropout(rate=0.3))
model.add(Dense(32, activation='relu'))
# model.add(Dropout(rate=0.3))
model.add(Dense(12, activation='relu'))
# model.add(Dropout(rate=0.3))
model.add(Dense(5, activation='relu'))
model.add(Dropout(rate=0.7))

model.add(Dense(8, activation='softmax'))
model.summary()

In [ ]:
es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
cp = ModelCheckpoint('fashion_cnn_1.h5')

In [ ]:
sgd = SGD(lr=0.0001, decay=1e-6, momentum= 0.9, nesterov = True)
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics = ['accuracy'])
model.summary()

In [ ]:
%%time
cnn_history = model.fit( X_train, 
                       y_train,                
                       batch_size=256,
                       epochs=50,
                       validation_data = (X_test, y_test),
                       callbacks = [es, cp],
                       verbose=2
                       )

In [ ]:
test_result = model.evaluate(X_test, y_test)
test_result

In [ ]:
print('test acc: ', round(test_result[1]*100, 2))

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1) ## 1행2열의 도표, 1열
plt.plot(cnn_history.history['accuracy'])
plt.plot(cnn_history.history['val_accuracy'])
plt.title('model acc')
plt.xlabel('epoch')
plt.ylabel('acc')
plt.legend(['train_acc','test_acc'])

plt.subplot(1, 2, 2) ## 1행2열의 도표, 2열
plt.plot(cnn_history.history['loss'])
plt.plot(cnn_history.history['val_loss'])
plt.title('model loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train_loss','test_loss'])